
파이선 병렬프로그래밍: 수치 해석 예제 실습
===================================================


#### 2024.10.
### 한국과학기술정보연구원 강지훈

***

### 필요 패키지

  - mpi4py
  - numpy
  - random
  - scikit-learn
  - matplotlib

***


# 1. 벡터와 행렬 연산 (I)

## 1.1. 행렬/벡터 만들기

In [ ]:
!mkdir examples

In [2]:
import numpy as np

np.set_printoptions(linewidth=np.inf)

n = 10

A = np.random.rand(n, n)
B = np.random.rand(n, n)
v = np.random.rand(n)
w = np.random.rand(n)

np.save("examples/A", A)
np.save("examples/B", B)
np.save("examples/v", v)
np.save("examples/w", w)


## 1.2. 벡터 내적

1. 순차코드
   
   <img src = "images/image01.png">

2. 병렬코드 - 등분할

    <img src = "images/image02.png">

In [ ]:
%%writefile examples/v.py
import numpy as np
from mpi4py import MPI

np.set_printoptions(linewidth=np.inf,precision=3)

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    v = np.load("examples/v.npy")
    w = np.load("examples/w.npy")
    n = v.size
else :
    v = None
    w = None
    n = 0

n = comm.bcast(n, root = 0)


##### n_local 크기 정하기 
n_local = int(n / size) # FIX ME

##### 분할된 n_local 크기만큼 배열 생성 
v_local = np.empty(n_local, dtype = np.float64)
w_local = np.empty(n_local, dtype = np.float64)

##### Scatter 함수로 벡터 분할
comm.Scatter(v, v_local, root = 0) # FIX ME
comm.Scatter(w, w_local, root = 0) # FIX ME

##### 프로세스별 Local sum 
s_local = np.dot(v_local, w_local) # FIX ME

##### reduce를 이용한 Global sum
s_global = comm.allreduce(s_local, MPI.SUM) # FIX ME

#if rank == 1:
print(rank, s_global)


In [ ]:
! mpiexec -np 2 python examples/v.py

3. 병렬코드 - 비등분할

    <img src = "images/image03.png">

In [ ]:
%%writefile examples/v_var.py
import numpy as np
from mpi4py import MPI

##### 시작점과 끝접의 인덱스를 반환
def para_range(n, size, rank) :
    iwork = divmod(n, size) 
    ista = rank * iwork[0] + min(rank, iwork[1])
    iend = ista + iwork[0] - 1
    if iwork[1] > rank :
        iend = iend + 1
    return ista, iend

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    v = np.load("examples/v.npy")
    w = np.load("examples/w.npy")
    n = v.size
else :
    v = None
    w = None
    n = 0

##### 벡터의 전체 크기를 broadcast
n = comm.bcast(n, root = 0)

##### 프로세스별 범위 할당
ista, iend = para_range(n, size, rank) # FIX ME
n_local = iend - ista + 1# FIX ME


##### Scatterv를 위해 n_local로부터 n_local_cnts 리스트 생성
n_local_cnts = comm.gather(n_local, root = 0)

v_local = np.empty(n_local, dtype = np.float64)
w_local = np.empty(n_local, dtype = np.float64)

##### n_local_cnts 리스트를 이용하여 Scatter로 벡터의 비균등 할당
comm.Scatterv((v, n_local_cnts), v_local, root = 0) #FIX ME
comm.Scatterv((w, n_local_cnts), w_local, root = 0) #FIX ME

##### 분할된 벡터의 내적
s_local = np.dot(v_local,w_local)

##### reduce를 이용한 Global sum
s_global = comm.reduce(s_local, MPI.SUM) #FIX ME

if rank == 0:
    print(n_local_cnts)
    print(s_global)


In [ ]:
! mpiexec -np 3 python examples/v_var.py

4. para_range 저장

In [ ]:
%%writefile examples/tools.py

def para_range(n, size, rank) :
    iwork = divmod(n, size) 
    ista = rank * iwork[0] + min(rank, iwork[1])
    iend = ista + iwork[0] - 1
    if iwork[1] > rank :
        iend = iend + 1
    return ista, iend


## 1.3. 행렬-벡터곱

1. 순차코드
   
    <img src = "images/image04.png">

In [ ]:
A = np.load("examples/A.npy")
v = np.load("examples/v.npy")

b = np.matmul(A,v)
print (b)


2. 행렬의 행 등분할

    <img src = "images/image05.png">

In [ ]:
%%writefile examples/Av.py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

##### 행렬 및 벡터 불러오기
##### 크기 n은 broadcast
if rank == 0 :
    A = np.load("examples/A.npy")
    v = np.load("examples/v.npy")
    n = v.size
    n = comm.bcast(n, root = 0)
else :
    A = None
    n = 0
    n = comm.bcast(n, root = 0)
    v = np.empty(n, dtype = np.float64)

##### n_local 크기 정하기 
n_local = int(n / size)

##### n_local 만큰 부분배열 선언
A_local = np.empty((n_local, n), dtype = np.float64)

##### 행렬의 행 분할
comm.Scatter(A, A_local, root = 0) #FIX ME

##### 벡터 v는 broadcast
comm.Bcast(v, root = 0)

##### 분할된 행렬과의 연산
b = np.matmul(A_local, v) #FIX ME

print(b, rank)

In [ ]:
! mpiexec -np 2 python examples/Av.py

3. 행렬의 행 비등분할

    <img src = "images/image06.png">

In [ ]:
%%writefile examples/Avar.py

# Matrix A의 Row decomposition

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

##### 행렬 및 벡터 불러오기
##### 크기 n은 broadcast
if rank == 0 :
    A = np.load("examples/A.npy")
    v = np.load("examples/v.npy")
    n = v.size
    n = comm.bcast(n, root = 0)

else :
    A = None
    n = 0
    n = comm.bcast(n, root = 0)
    v = np.empty(n, dtype = np.float64)

##### 프로세스별 범위 할당
ista, iend = para_range(n, size, rank)

##### 프로세스별 행수 설정
n_local = (iend - ista + 1)

##### 분할된 행렬 선언
A_local = np.empty((n_local, n), dtype = np.float64)

##### Scatterv를 위해 n_local를 이용한 리스트 생성. 이 때 부분행렬 행수가 아닌 전체 크기를 계산 
n_local_chunks = comm.gather(n_local * n, root = 0) #FIX ME

##### 행렬의 행 분할
comm.Scatterv((A, n_local_chunks), A_local, root = 0) #FIX ME

comm.Bcast(v, root = 0)

b = np.matmul(A_local,v)

print(b, rank)

In [ ]:
! mpiexec -np 3 python examples/Avar.py

4. 행렬/벡터의 행 비등분할

    <img src = "images/image07_1.png">

- $w_0$계산을 위해 각 프로세스가 소유한 $v$들이 필요 
  
  <img src = "images/image07_2.png">

- 각 랭크가 가진 분할된 벡터를 다른 모든 랭크로 보내서 행렬-벡터 곱을 수행해야함

    <img src = "images/image07_4.png">

- 단점 
  - 주고 받는 랭크 번호와 통신 대상이 매번 달라지게 됨
  - 프로세스수가 많을 경우 통신 거리가 멀어지며, 통신성능 저하

- 최종 형태
  - 순환 형태로 통신을 구현
  - 매번 곱하는 행렬의 열범위를 정확하게 설정해야 함

    <img src = "images/image07_3.png">

In [ ]:
%%writefile examples/Av_var.py

# Matrix A의 Row decomposition

from tools import para_range
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

##### 행렬 및 벡터 불러오기
##### 크기 n은 broadcast
if rank == 0 :
    A = np.load("examples/A.npy")
    v = np.load("examples/v.npy")
    n = v.size

else :
    A = None
    v = None
    n = 0

n = comm.bcast(n, root = 0)

##### 프로세스별 범위 할당
ista, iend = para_range(n, size, rank)

##### 프로세스별 행수 설정
n_local = (iend - ista + 1)

##### 분할된 행렬 선언
A_local = np.empty((n_local, n), dtype = np.float64)
v_local = np.empty(n_local, dtype = np.float64)

##### Scatterv를 위해 n_local를 이용한 리스트 생성. 이 때 부분행렬 행수가 아닌 전체 크기를 계산 
##### 분할된 벡터의 크기는 모든 프로세스가 알고 있어야 하므로 allgather를 이용
n_local_chunks = comm.gather(n_local * n, root = 0) #FIX ME
n_local_cnts = comm.allgather(n_local) #FIX ME

##### 행렬과 벡터 분할
comm.Scatterv([A, n_local_chunks], A_local, root = 0) #FIX ME
comm.Scatterv([v, n_local_cnts], v_local, root = 0) #FIX ME

##### 분할된 벡터 곱 범위 지정
col_idx_sta = []
col_idx_end = []

##### i번째 랭크가 곱해질 열 위치의 시작점과 끝점을 리스트 형태로 저장. 모든 랭크들이 계산
for i in range(size) :
    col_idx_sta.append(sum(n_local_cnts[:i]))
    col_idx_end.append(sum(n_local_cnts[:i])+n_local_cnts[i])

##### Local MV (최초 자신의 벡터부분)
b = np.matmul(A_local[:,col_idx_sta[rank]:col_idx_end[rank]], v_local)

##### 송수신 프로세스 지정
inext = rank + 1 if rank < size - 1 else 0
iprev = rank - 1 if rank > 0 else size - 1

##### (전체 프로세스 크기 -1) 번만큼 통신을 수행하고 부분행렬-부분벡터 곱을 수행
for i in range(size - 1) :
##### 몇 번째 이전 랭크로부터 받았는지를 iloc을 이용하여 확인
    iloc = iprev - i if iprev >= i else iprev - i + size
    v_recv = np.empty(n_local_cnts[iloc], dtype = np.float64)
##### 다음 랭크로 부분 벡터를 전달하며, 이전 랭크로부터 부분 벡터를 받음
    comm.Sendrecv(v_local, inext, 1, v_recv, iprev, 1) #FIX ME
##### 받은 벡터는 복사하여, 행렬-벡터 곱에 사용하는 한편, 다음 랭크로 전달할 수 있게 함
    v_local = np.copy(v_recv)
    b += np.matmul(A_local[:,col_idx_sta[iloc]:col_idx_end[iloc]], v_local)

print(b, rank)

In [ ]:
! mpiexec -np 3 python examples/Av_var.py